In [1]:
import warnings
warnings.simplefilter("ignore")


In [523]:
import pandas as pd
import psycopg2

# Criar conexão com PostgreSQL
conn = psycopg2.connect(
    host="localhost",
    dbname="bdmain",
    user="root",
    password="derek1708",
    port="5432"
)

In [524]:
query = 'SELECT tablename FROM pg_tables WHERE schemaname = \'public\';'
lista_tabela = pd.read_sql(query, conn)
print(lista_tabela)

            tablename
0            CLIENTES
1             EMPREGO
2         ESTADOCIVIL
3              FIADOR
4           HABITACAO
5   HISTORICO_CREDITO
6       INVESTIMENTOS
7        OUTROSFINANC
8           PROFISSAO
9           PROPOSITO
10            CREDITO


In [525]:
query = 'SELECT * FROM "CREDITO";'
dados = pd.read_sql(query, conn, index_col='IDCREDITO')
dados.head()

,Duracao,IDHISTCRED,IDPROPOSITO,Valor,IDINVESTIMENTOS,IDEMPREGO,TempoParcelamento,IDESTADOCIVIL,IDFIADOR,ResidenciaDesde,Idade,IDOUTROSFINANC,IDHABITACAO,EmprestimoExistente,IDPROFISSAO,Dependentes,SocioEmpresa,Estrangeiro,Status
IDCREDITO,,,,,,,,,,,,,,,,,,,
2,48,2,1,5951,2,3.0,2,1,3,2.0,22,3,1.0,1,4,1,0,0,0
3,12,1,2,2096,2,4.0,2,4,3,3.0,49,3,1.0,1,3,2,0,0,1
4,42,2,3,7882,2,4.0,2,4,4,4.0,45,3,2.0,1,4,2,0,0,1
5,24,3,4,4870,2,3.0,3,4,3,4.0,53,3,2.0,2,4,2,0,0,0
6,36,2,2,9055,1,3.0,2,4,3,4.0,35,3,2.0,1,3,2,1,0,1


In [526]:
dados.isna().sum()

Duracao                 0
IDHISTCRED              0
IDPROPOSITO             0
Valor                   0
IDINVESTIMENTOS         0
IDEMPREGO              10
TempoParcelamento       0
IDESTADOCIVIL           0
IDFIADOR                0
ResidenciaDesde         7
Idade                   0
IDOUTROSFINANC          0
IDHABITACAO             9
EmprestimoExistente     0
IDPROFISSAO             0
Dependentes             0
SocioEmpresa            0
Estrangeiro             0
Status                  0
dtype: int64

In [527]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 2 to 565
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Duracao              1000 non-null   int64  
 1   IDHISTCRED           1000 non-null   int64  
 2   IDPROPOSITO          1000 non-null   int64  
 3   Valor                1000 non-null   int64  
 4   IDINVESTIMENTOS      1000 non-null   int64  
 5   IDEMPREGO            990 non-null    float64
 6   TempoParcelamento    1000 non-null   int64  
 7   IDESTADOCIVIL        1000 non-null   int64  
 8   IDFIADOR             1000 non-null   int64  
 9   ResidenciaDesde      993 non-null    float64
 10  Idade                1000 non-null   int64  
 11  IDOUTROSFINANC       1000 non-null   int64  
 12  IDHABITACAO          991 non-null    float64
 13  EmprestimoExistente  1000 non-null   int64  
 14  IDPROFISSAO          1000 non-null   int64  
 15  Dependentes          1000 non-null   in

In [528]:
df_num = dados.select_dtypes(include=('float64')).columns
for c in df_num:
    dados[c].fillna(dados[c].median(), inplace=True)

In [530]:
dados.isna().sum()

Duracao                0
IDHISTCRED             0
IDPROPOSITO            0
Valor                  0
IDINVESTIMENTOS        0
IDEMPREGO              0
TempoParcelamento      0
IDESTADOCIVIL          0
IDFIADOR               0
ResidenciaDesde        0
Idade                  0
IDOUTROSFINANC         0
IDHABITACAO            0
EmprestimoExistente    0
IDPROFISSAO            0
Dependentes            0
SocioEmpresa           0
Estrangeiro            0
Status                 0
dtype: int64

In [533]:
dados.groupby('Status').size()

Status
0    300
1    700
dtype: int64

In [543]:
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


def train_xgboost(data: pd.DataFrame, target_column: str, test_size=0.2, random_state=42):
    """
    Treina um modelo XGBoost e retorna as métricas de desempenho em forma de tabela e o modelo treinado.
    
    Parâmetros:
        data (pd.DataFrame): Conjunto de dados incluindo features e target.
        target_column (str): Nome da coluna alvo.
        test_size (float): Proporção do conjunto de teste.
        random_state (int): Semente para reprodutibilidade.
    
    Retorna:
        pd.DataFrame: Métricas detalhadas do modelo.
        xgb.Booster: Modelo treinado.
    """
    # Separando features e alvo
    X = data.drop(columns=[target_column])
    y = data[target_column]
    
    # Divisão dos dados
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    # Calculando o scale_pos_weight para lidar com desbalanceamento
    neg, pos = y_train.value_counts()
    scale_pos_weight = neg / pos
    
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    # Conversão para DMatrix do XGBoost
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)
    
    # Parâmetros do modelo
    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'max_depth': 10,
        'eta': 0.1,
        'random_state': random_state,
        'scale_pos_weight': scale_pos_weight  # Ajuste para lidar com desbalanceamento
    }
    
    # Treinamento do modelo
    model = xgb.train(params, dtrain, num_boost_round=110)
    
    # Previsões
    y_pred_prob = model.predict(dtest)
    y_pred = np.round(y_pred_prob)
    
    # Cálculo das métricas
    class_report = classification_report(y_test, y_pred, output_dict=True)
    roc_auc = roc_auc_score(y_test, y_pred_prob)
    conf_matrix = confusion_matrix(y_test, y_pred)
    
    # Criando DataFrame com métricas detalhadas
    metrics_df = pd.DataFrame(class_report).transpose()
    metrics_df['roc_auc'] = roc_auc
    metrics_df["confusion_matrix"] = [conf_matrix.tolist()] * len(metrics_df)
    
    return metrics_df, model


In [544]:
metricas, modelo = train_xgboost(data=dados, target_column='Status', test_size=0.3)
metricas

,precision,recall,f1-score,support,roc_auc,confusion_matrix
0,0.636364,0.388889,0.482759,90.00,0.73582,"[[35, 55], [20, 190]]"
1,0.775510,0.904762,0.835165,210.00,0.73582,"[[35, 55], [20, 190]]"
accuracy,0.750000,0.750000,0.750000,0.75,0.73582,"[[35, 55], [20, 190]]"
macro avg,0.705937,0.646825,0.658962,300.00,0.73582,"[[35, 55], [20, 190]]"
weighted avg,0.733766,0.750000,0.729443,300.00,0.73582,"[[35, 55], [20, 190]]"
